## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [31]:
time_stamp, summary_table = f.dl_tf_pd('AAPL','5m','1mo', skip_dl=False)
print(f'{time_stamp}\n')
summary_table

2024-10-06 16:58:35



,model,prediction,prob_up,prob_static,prob_down,kelly_1_2,precision,recall,f1,support
0,XGBoost,static,0.125000,0.750000,0.125000,0.625000,0.749280,0.787879,0.768095,"[261.0, 330.0, 237.0]"
1,GradientBoosting,down,0.001483,0.462475,0.536042,0.304063,0.730769,0.721519,0.726115,"[261.0, 330.0, 237.0]"
2,RandomForest,down,0.023031,0.463849,0.513120,0.269680,0.756637,0.721519,0.738661,"[261.0, 330.0, 237.0]"
3,KNN,static,0.052939,0.543319,0.403742,0.314979,0.617925,0.793939,0.694960,"[261.0, 330.0, 237.0]"


In [34]:
col_headers = summary_table.model.tolist()
col_headers

['XGBoost', 'GradientBoosting', 'RandomForest', 'KNN']

In [35]:
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
summary_table_transposed

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,down,static
prob_up,0.125,0.001483,0.023031,0.052939
prob_static,0.75,0.462475,0.463849,0.543319
prob_down,0.125,0.536042,0.51312,0.403742
kelly_1_2,0.625,0.304063,0.26968,0.314979
precision,0.74928,0.730769,0.756637,0.617925
recall,0.787879,0.721519,0.721519,0.793939
f1,0.768095,0.726115,0.738661,0.69496
support,"[261.0, 330.0, 237.0]","[261.0, 330.0, 237.0]","[261.0, 330.0, 237.0]","[261.0, 330.0, 237.0]"


In [22]:
summary_table.reset_index(drop=True)

,model,prediction,prob_up,prob_static,prob_down,kelly_1_2,precision,recall,f1,support
0,XGBoost,static,0.1250,0.7500,0.1250,0.6250,0.7493,0.7879,0.7681,"[261.0, 330.0, 237.0]"
1,RandomForest,down,0.0015,0.4625,0.5360,0.3041,0.7566,0.7215,0.7387,"[261.0, 330.0, 237.0]"
2,GradientBoosting,static,0.0529,0.5433,0.4037,0.3150,0.7451,0.8061,0.7744,"[261.0, 330.0, 237.0]"
3,KNN,down,0.0230,0.4638,0.5131,0.2697,0.6882,0.5401,0.6052,"[261.0, 330.0, 237.0]"


In [30]:
summary_table.T.style.hide(axis='columns')

Matplotlib is building the font cache; this may take a moment.


model,XGBoost,RandomForest,GradientBoosting,KNN
prediction,static,down,static,down
prob_up,0.125000,0.001500,0.052900,0.023000
prob_static,0.750000,0.462500,0.543300,0.463800
prob_down,0.125000,0.536000,0.403700,0.513100
kelly_1_2,0.625000,0.304100,0.315000,0.269700
precision,0.749300,0.756600,0.745100,0.688200
recall,0.787900,0.721500,0.806100,0.540100
f1,0.768100,0.738700,0.774400,0.605200
support,"[261.0, 330.0, 237.0]","[261.0, 330.0, 237.0]","[261.0, 330.0, 237.0]","[261.0, 330.0, 237.0]"


In [26]:
list(summary_table.columns)

['model',
 'prediction',
 'prob_up',
 'prob_static',
 'prob_down',
 'kelly_1_2',
 'precision',
 'recall',
 'f1',
 'support']

In [46]:
stock_df = f.load('NVDA', '5m')
stock_df.sample(7)

,open,high,low,close,adj_close,volume,dividends,stock_splits
Datetime,,,,,,,,
2024-09-06 09:15:00-04:00,108.120000,108.640000,108.090000,108.383000,108.383000,0,0.0,0.0
2024-09-25 14:45:00-04:00,123.417503,123.599998,123.400002,123.541000,123.541000,1383046,0.0,0.0
2024-09-19 09:05:00-04:00,116.950000,117.000000,116.801000,116.980200,116.980200,0,0.0,0.0
2024-09-20 07:40:00-04:00,116.850000,117.070000,116.801000,117.040000,117.040000,0,0.0,0.0
2024-09-25 13:45:00-04:00,122.923897,123.000000,122.620003,122.789803,122.789803,2943278,0.0,0.0
2024-09-24 19:25:00-04:00,121.780000,121.830000,121.770000,121.810000,121.810000,0,0.0,0.0
2024-10-01 13:50:00-04:00,116.650002,116.849998,116.559998,116.839996,116.839996,1532489,0.0,0.0


In [47]:
stock_df = stock_df.reset_index().rename(columns={'Datetime': 'date',
                                                  'Date': 'date',
                                                 })
stock_df.sample(7)

,date,open,high,low,close,adj_close,volume,dividends,stock_splits
3012,2024-09-26 15:00:00-04:00,123.870003,123.872398,123.529999,123.682701,123.682701,2069733,0.0,0.0
3276,2024-09-30 05:00:00-04:00,119.470000,119.480000,119.300000,119.340000,119.340000,0,0.0,0.0
611,2024-09-10 06:55:00-04:00,106.330000,106.440000,106.290000,106.350000,106.350000,0,0.0,0.0
928,2024-09-11 17:20:00-04:00,116.510000,116.910000,116.390000,116.510000,116.510000,0,0.0,0.0
2478,2024-09-23 18:30:00-04:00,116.035000,116.040000,116.020000,116.030000,116.030000,0,0.0,0.0
1508,2024-09-16 17:40:00-04:00,116.550000,116.600000,116.480000,116.501300,116.501300,0,0.0,0.0
3316,2024-09-30 08:20:00-04:00,118.150700,118.190000,117.830000,117.900000,117.900000,0,0.0,0.0


### Check Summary Output

In [ ]:
symbol='NVDA'
interval='5m'
period='1mo'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval, period)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')